# Detección de Caras con Haar

In [8]:
%matplotlib

# Importamos las librerías necesarias
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import time


Using matplotlib backend: Qt5Agg


In [3]:
# Leemos los clasificadores pre-entrenados (buscar dentro del path de anaconda, colocar barras dobles)
#--------------------------------------------------------------------------------
face_cascade = cv.CascadeClassifier('C:\\Users\\Lautaro\\anaconda3\\pkgs\\libopencv-4.0.1-hbb9e17c_0\\Library\\etc\\haarcascades\\haarcascade_frontalface_default.xml')
eye_cascade = cv.CascadeClassifier('C:\\Users\Lautaro\\anaconda3\\pkgs\\libopencv-4.0.1-hbb9e17c_0\\Library\\etc\\haarcascades\\haarcascade_eye.xml')
smile_cascade = cv.CascadeClassifier('C:\\Users\\Lautaro\\anaconda3\\pkgs\\libopencv-4.0.1-hbb9e17c_0\\Library\\etc\\haarcascades\\haarcascade_smile.xml')


In [11]:
# Selección de ROI
def SelectROI(cap):
    global frame
    ret, frame = cap.read()
    showCrosshair = False
    fromCenter = False
    r = cv.selectROI(frame, fromCenter, showCrosshair)
    roi = frame[int(r[1]):int(r[1]+r[3]),int(r[0]):int(r[0]+r[2])]
    cv.imshow("Image", roi)
    cv.waitKey(0)
    track_window = (r[1], r[0], r[3], r[2])
    return roi, track_window

cap = cv.VideoCapture(0)
roi_face, track_window_face = SelectROI(cap)
roi_eye, track_window_eye = SelectROI(cap)
roi_smile, track_window_smile = SelectROI(cap)

In [12]:
print(track_window_face)
print(track_window_eye)
print(track_window_smile)

(212, 251, 197, 179)
(272, 311, 38, 36)
(344, 321, 40, 77)


In [22]:
cap = cv.VideoCapture(0)
while(1):
    ret, frame = cap.read()
    if ret == True:
        grayImage = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        grayImage = cv.equalizeHist(grayImage)
        # Llamamos al clasificador de Haar (AdaBoost)
        #--------------------------------------------
        faces = face_cascade.detectMultiScale(grayImage, 1.01, 10, minSize=(180, 160), maxSize=(210, 190))
        #, minSize= (100,130),maxSize=(200,400))

        # Recorro las caras encontradas
        #------------------------------
        for (x,y,w,h) in faces:
            # Le dibujamos un rectángulo celeste
            cv.rectangle(frame,(x,y),(x+w,y+h),(255,255,0),2)
            # Definimos las ROIs en la imagen gris y color
            roi_gray = grayImage[y:y+h, x:x+w] 
            roi_color = frame[y:y+h, x:x+w]
            # Dibujamos la sonrisa con cuadrado rojo
            smile = smile_cascade.detectMultiScale(roi_gray, 1.01, 2,  minSize=(20, 50), maxSize=(150, 150))
            for j, (sx, sy, sw, sh) in enumerate(smile):
                if j>1:
                    break;
                cv.rectangle(roi_color,(sx,sy),(sx+sw,sy+sh),(0,0,255),2)
            # Para cada rostro hallado le buscamos los ojos
            eyes = eye_cascade.detectMultiScale(roi_gray, minSize=(30, 30), maxSize=(50, 50))
            # En los ojos hallados les dibujamos rectángulos
            for i, (ex,ey,ew,eh) in enumerate(eyes):
                if i>1:
                    break;
                cv.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
                
        cv.imshow('Seguimiento',frame)
        time.sleep(0.01)
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break
    else:
        break
cv.destroyAllWindows()
cap.release()